# Binary Credit Loan Decisions with Unfairness Mitigation

#### Mitigating accuracy disparities based on equalized odds

## Historical Context

## Concrete Harms

#### Denial of Resources (Wealth)

## Unmitigated Classifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from sklearn.

In [ ]:
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import (
    selection_rate,
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference
)

## Alternatives, Trade-offs, and Next Steps

### Fairness Under Unawareness